In [1]:
import datetime
import pandas as pd
import numpy as np

### 一.更新global_model_mean

In [2]:
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
add_combine_detail = pd.read_csv('../tmp/train/add_new_detail.csv')
car_autohome_all = car_autohome_all.loc[~(car_autohome_all['detail_slug'].isin(list(add_combine_detail.detail_slug.values))),:].reset_index(drop=True)

global_model_mean_part1 = pd.read_csv('../tmp/train/global_model_mean_part1.csv')
global_model_mean_part1['median_price'] = global_model_mean_part1['median_price'] / 0.96
global_model_mean_part1 = global_model_mean_part1.loc[global_model_mean_part1.groupby(['brand_slug', 'model_slug', 'online_year']).price_bn.idxmin(), :].reset_index(drop=True)
global_model_mean_part1 = global_model_mean_part1.drop_duplicates(['model_slug', 'online_year'])

global_model_mean_part2 = pd.read_csv('../tmp/train/global_model_mean_part2.csv')
global_model_mean = global_model_mean_part1.append(global_model_mean_part2,sort=False)

In [3]:
train = pd.read_csv('../tmp/train/train.csv')
div_price_bn_k_param = pd.read_csv('../tmp/train/div_price_bn_k_param.csv')
car_autohome_all = car_autohome_all.sort_values(by=['brand_slug', 'model_slug', 'online_year', 'price_bn']).reset_index(drop=True)
car_autohome_all['used_years'] = datetime.datetime.now().year - car_autohome_all['online_year']
car_autohome_all.loc[(car_autohome_all['used_years'] < 0), 'used_years'] = 0

# 上牌时间和上市时间相同
train = train.loc[(train['online_year'] == train['year']), :].reset_index(drop=True)

# 根据款型计算中位数
median_price = train.groupby(['brand_slug', 'brand_name', 'model_slug', 'model_name', 'detail_slug', 'online_year', 'price_bn'])['price'].median().reset_index().rename(columns={'price': 'median_price'})
median_price = median_price.sort_values(by=['brand_slug', 'model_slug', 'online_year', 'price_bn']).reset_index(drop=True)

# 取低配数据
low_config_car = median_price.loc[median_price.groupby(['brand_slug', 'model_slug', 'online_year']).price_bn.idxmin(), :].reset_index(drop=True)
low_config_car = low_config_car.drop_duplicates(['model_slug', 'online_year'])

# 调整指导价差,确保同条件下高配比低配价格高
part1 = pd.DataFrame()
for model_slug, online_year in low_config_car.loc[:, ['model_slug', 'online_year']].values:
    car_autohome_temp = car_autohome_all.loc[(car_autohome_all['model_slug'] == model_slug)&(car_autohome_all['online_year'] == online_year), :].reset_index(drop=True)
    car_autohome_temp = car_autohome_temp.merge(median_price.loc[:, ['detail_slug', 'median_price']], how='left', on=['detail_slug'])
    low_config_price, price_bn = car_autohome_temp.loc[(car_autohome_temp['median_price'].notnull()), ['median_price', 'price_bn']].values[0]
    used_years = car_autohome_temp.loc[0, 'used_years']
    k = div_price_bn_k_param.loc[(div_price_bn_k_param['used_years'] == used_years), ['k']].values[0]
    for i in range(0, len(car_autohome_temp)):
        car_autohome_temp.loc[i, 'median_price'] = float('%.2f' % ((car_autohome_temp.loc[i, 'price_bn'] - price_bn) * k + low_config_price))
    part1 = part1.append(car_autohome_temp, sort=False).reset_index(drop=True)
    
low_config_car = part1.loc[part1.groupby(['brand_slug', 'model_slug', 'online_year']).price_bn.idxmin(), :].reset_index(drop=True)
low_config_car = low_config_car.drop_duplicates(['model_slug', 'online_year'])

In [4]:
def update_price(df):
    update_price = low_config_car.loc[(low_config_car['detail_slug'] == df['detail_slug']),'median_price'].values
    if len(update_price) == 0:
        return df['median_price']
    else:
        return update_price[0]

global_model_mean['median_price'] = global_model_mean.apply(update_price, axis=1) 
global_model_mean.to_csv('../tmp/train/low_config_car_price.csv',index=False)

In [5]:
global_model_mean = pd.read_csv('../tmp/train/low_config_car_price.csv')

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
add_combine_detail = pd.read_csv('../tmp/train/add_new_detail.csv')
car_autohome_all = car_autohome_all.loc[~(car_autohome_all['detail_slug'].isin(list(add_combine_detail.detail_slug.values))),:].reset_index(drop=True)

div_price_bn_k_param = pd.read_csv('../tmp/train/div_price_bn_k_param.csv')
car_autohome_all = car_autohome_all.sort_values(by=['brand_slug', 'model_slug', 'online_year', 'price_bn']).reset_index(drop=True)
car_autohome_all['used_years'] = datetime.datetime.now().year - car_autohome_all['online_year']
car_autohome_all.loc[(car_autohome_all['used_years'] < 0), 'used_years'] = 0

# 调整指导价差,确保同条件下高配比低配价格高
result = pd.DataFrame()
for model_slug, online_year in global_model_mean.loc[:, ['model_slug', 'online_year']].values:
    car_autohome_temp = car_autohome_all.loc[(car_autohome_all['model_slug'] == model_slug)&(car_autohome_all['online_year'] == online_year), :].reset_index(drop=True)
    car_autohome_temp = car_autohome_temp.merge(global_model_mean.loc[:, ['detail_slug', 'median_price']], how='left', on=['detail_slug'])
    low_config_price, price_bn = car_autohome_temp.loc[(car_autohome_temp['median_price'].notnull()), ['median_price', 'price_bn']].values[0]
    used_years = car_autohome_temp.loc[0, 'used_years']
    k = div_price_bn_k_param.loc[(div_price_bn_k_param['used_years'] == used_years), ['k']].values[0]
    for i in range(0, len(car_autohome_temp)):
        car_autohome_temp.loc[i, 'median_price'] = float('%.2f' % ((car_autohome_temp.loc[i, 'price_bn'] - price_bn) * k + low_config_price))
    result = result.append(car_autohome_temp, sort=False).reset_index(drop=True)

### 二.保留高配价格

In [6]:
def retain_high_config(df):
    normal_update_price = median_price.loc[(median_price['detail_slug'] == df['detail_slug']),'median_price'].values
    if len(normal_update_price) == 0:
        return df['median_price']
    
    all_update_price = np.median(origin_train.loc[(origin_train['detail_slug'] == df['detail_slug']),'price'])
    if normal_update_price[0] >= all_update_price:
        update_price = all_update_price
    else:
        update_price = normal_update_price[0]
    return update_price
#     if df['median_price'] < update_price:
#         return update_price
#     return df['median_price'] 

combine_detail = pd.read_csv('../tmp/train/combine_detail.csv', low_memory=False)
add_combine_detail = pd.read_csv('../tmp/train/add_new_detail.csv')
combine_detail = combine_detail.loc[~(combine_detail['car_autohome_detail_id'].isin(list(add_combine_detail.detail_slug.values))),:].reset_index(drop=True)


origin_train = pd.read_csv('../tmp/train/train.csv')
# 上牌时间和上市时间相同
train = origin_train.copy()
train = train.loc[(train['online_year'] == train['year']), :].reset_index(drop=True)
# 根据款型计算中位数
median_price = train.groupby(['brand_slug', 'brand_name', 'model_slug', 'model_name', 'detail_slug', 'online_year', 'price_bn'])['price'].median().reset_index().rename(columns={'price': 'median_price'})
median_price = median_price.sort_values(by=['brand_slug', 'model_slug', 'online_year', 'price_bn']).reset_index(drop=True)

# 保留高配价格
retain_high_config_price = result.copy()
retain_high_config_price['median_price'] = retain_high_config_price.apply(retain_high_config,axis=1)
model_online = retain_high_config_price.drop_duplicates(['model_slug', 'online_year'])

# 调整指导价差,确保同条件下高配比低配价格高
result_retain = pd.DataFrame()
for model_slug, online_year in model_online.loc[:, ['model_slug', 'online_year']].values:
    car_autohome_temp = retain_high_config_price.loc[(retain_high_config_price['model_slug'] == model_slug)&(retain_high_config_price['online_year'] == online_year), :].reset_index(drop=True)
    used_years = car_autohome_temp.loc[0, 'used_years']
    k = div_price_bn_k_param.loc[(div_price_bn_k_param['used_years'] == used_years), ['k']].values[0]
    for i in range(0, len(car_autohome_temp)-1):
        if car_autohome_temp.loc[i, 'median_price'] > car_autohome_temp.loc[i+1, 'median_price']:
            div_price_bn = car_autohome_temp.loc[i+1, 'price_bn'] - car_autohome_temp.loc[i, 'price_bn']
            car_autohome_temp.loc[i+1, 'median_price'] = float('%.3f' % (div_price_bn * k + car_autohome_temp.loc[i, 'median_price']))
    result_retain = result_retain.append(car_autohome_temp, sort=False).reset_index(drop=True)

In [7]:
final = result_retain.copy()
final = final.sort_values(by=['brand_slug', 'model_slug', 'online_year', 'price_bn']).reset_index(drop=True)
final = final.merge(combine_detail.loc[:, ['detail_model_slug', 'car_autohome_detail_id']].rename(columns={'car_autohome_detail_id':'detail_slug'}), how='left', on=['detail_slug'])
final = final.loc[(final['detail_model_slug'].notnull()), :]
final = final.drop(['volume_extend','emission_standard','volume','control'],axis=1)
final.to_csv('../tmp/train/global_model_mean.csv', index=False)
print('最终组合款型数:', len(final))

最终组合款型数: 31990


### 三.确认同系同款高车龄价格低

In [29]:
verify_price = pd.read_csv('../tmp/train/global_model_mean.csv',low_memory=False)
columns = list(verify_price.columns)
low_config_car = verify_price.loc[verify_price.groupby(['brand_slug', 'model_slug', 'online_year']).price_bn.idxmin(), :].reset_index(drop=True)
model = low_config_car.drop_duplicates(['model_slug'])

# 查找异常
price_exception = pd.DataFrame()
for model_slug in model.loc[:, ['model_slug']].values:
    car_autohome_temp = low_config_car.loc[(low_config_car['model_slug'] == model_slug[0]), :].reset_index(drop=True)
    for i in range(0, len(car_autohome_temp)-1):
        if car_autohome_temp.loc[i,'median_price'] > car_autohome_temp.loc[i+1,'median_price']:     
            temp = car_autohome_temp.loc[i, :].copy()
            temp['next_year_price'] = car_autohome_temp.loc[i+1,'median_price']
            price_exception = price_exception.append(temp, sort=False).reset_index(drop=True)

columns.append('next_year_price')
price_exception = price_exception.loc[:,columns]
price_exception.to_csv('../tmp/train/verify_price.csv',index=False)

In [16]:
result.loc[(result['detail_slug'] == 27968),:]

,detail_slug,brand_name,brand_slug,model_name,model_slug,detail_name,manufacturer,price_bn,body,energy,volume_extend,emission_standard,listed_year,online_year,volume,control,used_years,median_price
13408,27968,纳智捷,130,优6 SUV,3293,2017款 1.8T 时尚升级型,东风裕隆,14.98,紧凑型SUV,汽油,L,国IV(国V),2016,2017,1.8,自动,2,9.71


In [17]:
retain_high_config_price.loc[(retain_high_config_price['detail_slug'] == 27968),:]

,detail_slug,brand_name,brand_slug,model_name,model_slug,detail_name,manufacturer,price_bn,body,energy,volume_extend,emission_standard,listed_year,online_year,volume,control,used_years,median_price
13408,27968,纳智捷,130,优6 SUV,3293,2017款 1.8T 时尚升级型,东风裕隆,14.98,紧凑型SUV,汽油,L,国IV(国V),2016,2017,1.8,自动,2,8.5


In [18]:
result_retain.loc[(result_retain['detail_slug'] == 27968),:]

,detail_slug,brand_name,brand_slug,model_name,model_slug,detail_name,manufacturer,price_bn,body,energy,volume_extend,emission_standard,listed_year,online_year,volume,control,used_years,median_price
13408,27968,纳智捷,130,优6 SUV,3293,2017款 1.8T 时尚升级型,东风裕隆,14.98,紧凑型SUV,汽油,L,国IV(国V),2016,2017,1.8,自动,2,9.706


In [153]:
train = pd.read_csv('../tmp/train/train.csv')
detail_slug = 19247
train = train.loc[(train['online_year'] == train['year']), :].reset_index(drop=True)
print(np.median(list(train.loc[(train['detail_slug'] == detail_slug),'price'])))
train.loc[(train['detail_slug'] == detail_slug),:]

70.0


,source_id,origin_name,year,month,mile,price,city,domain,transfer_owner,title,brand_slug,model_slug,detail_slug,online_year,price_bn,brand_name,model_name,detail_name,province
22716,3007157804,奔驰S级 2014款 S 320 L 豪华型(进口),2014,7.0,8.9,67.9,邯郸,guazi.com,0.0,奔驰s级2014款s320l豪华型(进口),36.0,59.0,19247.0,2014.0,110.8,奔驰,奔驰S级,2014款 S 320 L 豪华型,河北
22717,3008200226,奔驰S级 2014款 S 320 L 豪华型(进口),2014,11.0,4.6,70.0,上海,guazi.com,0.0,奔驰s级2014款s320l豪华型(进口),36.0,59.0,19247.0,2014.0,110.8,奔驰,奔驰S级,2014款 S 320 L 豪华型,上海
22718,3003451776,奔驰S级 2014款 S 320 L 豪华型(进口),2014,10.0,6.3,70.0,秦皇岛,guazi.com,0.0,奔驰s级2014款s320l豪华型(进口),36.0,59.0,19247.0,2014.0,110.8,奔驰,奔驰S级,2014款 S 320 L 豪华型,河北


In [154]:
final = pd.read_csv('../tmp/train/global_model_mean.csv')
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34582 entries, 0 to 34581
Data columns (total 15 columns):
detail_slug          34582 non-null int64
brand_name           34582 non-null object
brand_slug           34582 non-null int64
model_name           34582 non-null object
model_slug           34582 non-null int64
detail_name          34582 non-null object
manufacturer         34582 non-null object
price_bn             34582 non-null float64
body                 34582 non-null object
energy               34582 non-null object
listed_year          34582 non-null int64
online_year          34582 non-null int64
used_years           34582 non-null int64
median_price         34582 non-null float64
detail_model_slug    34582 non-null object
dtypes: float64(2), int64(6), object(7)
memory usage: 4.0+ MB
